In [ ]:
import pandas as pd
import os

In [ ]:
def getheaders():
    test_headers = list(pd.read_csv(test, nrows = 0).columns)
    train_headers = list(pd.read_csv(train, nrows = 0).columns)
    if os.path.isfile('properties.txt'):
        with open('properties.txt', "r") as f:
            properties = f.read().splitlines()
        check = [item for item in list(set().union(test_headers,train_headers)) if item not in properties]
        if len(check) != 0:
            raise Exception("lists are not the same")
    else:
        properties = list(set().union(test_headers,train_headers))
        file = open('properties.txt', "w")
        for item in properties:
            print(item, file = file)
        file.close()
    return properties

In [ ]:
def get_letor_row(df, prop_id, srch_id, entry):
    target = prop_id.iloc[entry][0]
    qid = "qid:" + str(srch_id.iloc[entry][0])
    features = []
    for i, feature in enumerate(properties):
        try: 
            value = df.loc[entry, feature]
        except:
            pass
        else:
            features.append(str(i + 1) + ":" + str(value))
    features = " ".join(features)
    return " ".join([str(target),qid,features])

In [ ]:
def open_set(hotel_set_file):
    hotel_set = pd.read_csv(hotel_set_file)
    hotel_set.drop(hotel_set.select_dtypes(['object']), inplace=True, axis=1)
    hotel_set_prop_id = hotel_set[['prop_id']]
    hotel_set_srch_id = hotel_set[['srch_id']]
    hotel_set.drop(hotel_set.select_dtypes(['object']), inplace=True, axis=1)
    hotel_set_type = hotel_set_file.split('_')[1][:-3]
    file = open("../LambdaMart/%s.txt" % hotel_set_type, "w")
    for entry in range(len(hotel_set)):
        print(get_letor_row(hotel_set, hotel_set_prop_id, hotel_set_srch_id, entry), file=file)
    file.close()

In [ ]:
test = 'hoteltest_testing_set.csv'
train = 'hoteltest_training_set.csv'

In [ ]:
properties = getheaders()